# 1 - Import das bibliotécas utilizadas na etapa de avaliação de hiperparametros e do desempenho de diversos modelos treinados

In [2]:
#imports que serão usados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, plot_roc_curve
from sklearn.metrics import classification_report
import scikitplot as skplt
pd.set_option('display.max_colwidth', -1)

# 2 - Import do dataframe pós tratamento e limpeza dos dados. 

In [10]:
df = pd.read_csv('df.csv')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   int64  
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   Gender                   4410 non-null   object 
 8   JobLevel                 4410 non-null   int64  
 9   JobRole                  4410 non-null   object 
 10  MaritalStatus            4410 non-null   object 
 11  MonthlyIncome            4410 non-null   int64  
 12  NumCompaniesWorked       4391 non-null   float64
 13  PercentSalaryHike        4410 non-null   int64  
 14  StockOptionLevel        

### 3) Construção do modelo de Arvore de Decisões

Para avaliar o problema proposto de attrition, além da etapa de exploração de dados, onde conseguimos visualizar diretamente os principais fatores que influenciam attrition, também utilizaremos o modelo de Arvore de Decisões para classificar os funcionários que potencialmente podem deixar a companhia de forma voluntária. O objetivo do modelo é criar um processo preditivo que auxilie a tomada de decisão e a dê visibilidade para os potenciais casos de attrition. 

Além disso, um dos propósitos é também entender como o modelo da Arvore de Decisões se comporta em relação ao problema proposto (attrition), portanto, além da melhor precisão, também serão avaliados a otimização dos hiperparâmetros do modelo preditivo, utilizando-se a bibliotéca a função GridSearchCV da bibliotéca Scikit-learn. 

##### 3.1 - Métricas de desempenho

Para avaliação do modelo de DecisionTree, utilizaremos algumas métricas de desempenho de modelos de classificação.

Entre elas, temos: Precisão, Revocação, F-Medida e Acurácia

In [8]:
#Funções das métricas de desempenho

# Matriz de Confusão
def Matriz_Confusao (p_y_test, p_y_pred):
    vn, fp, fn, vp = confusion_matrix(p_y_test, p_y_pred).ravel()
    return vn, fp, fn, vp

#Acuracia
def calcacuracia (vp,fp,vn,fn):
    acuracia = (vp+vn)/(vp+fp+vn+fn)
    return acuracia

#Precisao
def calcprecisao(vp,fp,vn,fn):
    precisao = vp / (vp + fp)
    return precisao

#Revocação
def calcrevocacao(vp,fp,vn,fn):
    revocacao = vp / (vp + fn)
    return revocacao

#F-Medida
def calcfmedida(P,R):
    fmedida = 2 * (P * R / (P + R))
    return fmedida


# Utilizando as funções de report do skplt.metrics
def resultados(res):

    # Medidas de acerto 
    for phase in res.keys():
        print("-------------------------------------------------------------")
        print("Evaluating %s" %(phase))
        print(classification_report(res[phase]["actual"], res[phase]["pred"]))
    
    # Matriz de Confusão
    for phase in res.keys():
        print("-------------------------------------------------------------")
        print("Evaluating %s" %(phase))
        skplt.metrics.plot_confusion_matrix(res[phase]["actual"], res[phase]["pred"])
        plt.show()
    
    # Curva ROC-AUC
    for phase in res.keys():
        print("-------------------------------------------------------------")
        print("Evaluating %s" %(phase))
        skplt.metrics.plot_roc_curve(res[phase]["actual"], res[phase]["prob"])
        plt.show()

    return

##### 3.2 - Avaliação de modelos

Foi observado que alguns trabalhos na literatura, realizam a construção dos modelos separando os datasets por diferentes departamentos da companhia, visto que diferentes departamentos podem possuir caracteristicas totalmente diferentes em relação a vários fatores de performance, satisfação, workload, entre outros. 

Seguindo esta recomendação, serão separados os datasets em três cenários. 

O primeiro cenário tem como propósito direcionar a construção de um modelo que atenda a necessidade de toda a companhia. Portanto, serão utilizados todos os dados. O segundo e o terceiro cenário, utilizaram respectivamente os dados apenas do departamento de Sales e de R&D, visto que são os departamentos que possuem maior volume de dados. O departamento de RH possui uma quantidade muito pequena de dados, portanto, não faria sentido construir um modelo específico para o departamento. 


In [12]:
#Separação dos dados para train/test de diferentes modelos, considerando: Dataset inteiro e separado por dois departamentos

X_Sales = df[(df['Department']=='Sales')][df.columns.drop('Attrition')]
y_sales = df[(df['Department']=='Sales')][['Attrition']]

X_RandD = df[(df['Department']=='Research & Development')][df.columns.drop('Attrition')]
y_RandD = df[(df['Department']=='Research & Development')][['Attrition']]

X_all = df[df.columns.drop('Attrition')]
y_all = df[['Attrition']]

Para otimizar o processo de avaliação dos melhores hiperparametros da Arvore de Descisão, construímos um algoritmo utilizando Pipeline e Gridsearch, o qual realiza um loop através de um conjunto de combinações de diferentes diferentes parâmetros, tanto no pré-processamento (estratégia de tratamento de valores nulos), como nos hiperparâmetros da Arvore de Descisão.

##### 3.2.1 - Primeiro batch de testes com diferentes hiperparametros

Em testes iniciais, verificamos que quando as diferentes ranges de max_depth da Arvore de Descisão eram passados diretamente no GridSearch, estavam sendo selecionados apenas o resultado maior de max_depth, o que deixava o modelo overfitado. Portanto, nas interações criadas, forçamos o GridSearch a sempre adotar um valor específico de max_depth, que foi interado com todas as outras possibilidades de escolha dos outros parâmetros. 

OBS: O código abaixo está comentado, pois demorou aproximandamente 24 horas para executar, o que acabou tirando um pouco o propósito da tentativa de otimizar o processo, mas nos ajudou a afunilar as escolhas e os ranges para uma próxima rodada de testes

In [ ]:
''''
test_size = [0.15,0.20,0.25]
SIStrategyCat = 'most_frequent'
SIStrategyNum = ['mean','median','most_frequent']
OneHotStrat = 'ignore'
GSscoring=['precision', 'recall','f1']
max_depth=[[5],[10],[20],[25],[40],[50],[75],[100]]
train_test_data = [[X_all,y_all],[X_Sales,y_sales],[X_RandD,y_RandD]]

#Nomeando os dataframes
X_all.name = 'X_all'
y_all.name = 'y_all'
X_Sales.name = 'X_Sales'
y_sales.name = 'y_sales'
X_RandD.name = 'X_RandD'
y_RandD.name = 'y_RandD'

            
#Separação das colunas numéricas continuas e categóricas que receberão diferentes tratativas no pipeline
numeric_features = df.drop(columns=['Attrition']).select_dtypes(exclude=[object]).columns.values.tolist()
categorical_features = df.select_dtypes(include=[object]).columns.values.tolist()

#Dicionário para armazenar os resultados
resumo_resultados = {
            'VP': [],
            'FP': [],
            'VN': [],
            'FN': [],
            'Acuracia'  : [],
            'Precisao'  : [], 
            'Revocação' : [],
            'F-Medida'  : [],
            'ParametrosArvore': [],
            'data':[],
            'Test_size': [],
            'SIStrategyNum': [],
            'OneHotStrat':[],
            'GSscoring':[],
            'max_depth':[]
            
                     }

for data in train_test_data:
    for size in test_size:
        for numstrat in SIStrategyNum:
            for scoringmethod in GSscoring:
                for depth in max_depth:
                    X_train, X_test, y_train, y_test = train_test_split(data[0],data[1], test_size=size, random_state=42)


                    cnts_pipeline = Pipeline([
                        ('impute', SimpleImputer(strategy=numstrat)),
                        ('scale', StandardScaler())])

                    categ_pipeline = Pipeline([
                        ('impute', SimpleImputer(strategy=SIStrategyCat)),
                        ('onehot', OneHotEncoder(handle_unknown=OneHotStrat))])

                    preprocess_pipeline = ColumnTransformer([
                        ('continuous', cnts_pipeline, numeric_features),
                        ('cat', categ_pipeline, categorical_features)])

                    pipeline = Pipeline(steps=[("preprocessor", preprocess_pipeline), ("classifier", DecisionTreeClassifier())])

                    param_grid = [{
                               'classifier__criterion': ['gini','entropy'],
                               'classifier__splitter': ['best', 'random'],
                               'classifier__max_depth': depth,
                               'classifier__max_features': ['auto', 'sqrt', 'log2'],
                               'classifier__min_samples_split':range(1,10),
                               'classifier__min_samples_leaf': range(1,10)}]

                    model_gs = GridSearchCV(pipeline, param_grid, cv=5, scoring=scoringmethod)
                    model_gs.fit(X_train, y_train)

                    print("________________________________________________________________")
                    print('Modelo/Dados =',data[0].name,data[1].name)
                    print("Combinação de Parâmetros:")
                    print('Test size =',size,'NumericInputer =',numstrat,'GSscoring =',scoringmethod, 'Max_depth =',depth)
                    print("*********************\n")
                    print("Resultados gerais:")
                    print('Best Score: ',model_gs.best_score_)
                    print('Melhores parâmetros da árvore: ',model_gs.best_params_)
                    print("*********************\n")
                    print("Matriz de Confusão:")
                    Results = {"train": {"actual": y_train,
                               "pred": model_gs.predict(X_train),
                               "prob": model_gs.predict_proba(X_train)},

                       "test": {"actual": y_test,
                               "pred": model_gs.predict(X_test),
                               "prob": model_gs.predict_proba(X_test)}}

                    resultados(Results)
                    print("________________________________________________________________")

                    y_pred = model_gs.predict(X_test)

                    vn, fp, fn, vp = Matriz_Confusao (y_test, y_pred)
                    Acuracia = calcacuracia(vp,fp,vn,fn)
                    P = calcprecisao(vp,fp,vn,fn)
                    R = calcrevocacao(vp,fp,vn,fn)
                    FMedida = calcfmedida(P,R)

                    resumo_resultados['VP'].append(vp)
                    resumo_resultados['FP'].append(fp)
                    resumo_resultados['VN'].append(vn)
                    resumo_resultados['FN'].append(fn)                
                    resumo_resultados['Acuracia'].append(Acuracia)
                    resumo_resultados['Precisao'].append(P)                
                    resumo_resultados['Revocação'].append(R)
                    resumo_resultados['F-Medida'].append(FMedida)                             
                    resumo_resultados['ParametrosArvore'].append(model_gs.best_params_)
                    resumo_resultados['data'].append([data[0].name,data[1].name])
                    resumo_resultados['Test_size'].append(size)
                    resumo_resultados['SIStrategyNum'].append(numstrat)
                    resumo_resultados['OneHotStrat'].append(OneHotStrat)
                    resumo_resultados['GSscoring'].append(scoringmethod)
                    resumo_resultados['max_depth'].append(depth)
                    

                    resultados_gerais = pd.DataFrame(resumo_resultados)

resultados_gerais.to_csv('resultados_gerais.csv', index = False)

In [5]:
resultados_exemplo1 = pd.read_csv('resultados_gerais.csv')
resultados_exemplo1.head(10)

,VP,FP,VN,FN,Acuracia,Precisao,Revocação,F-Medida,ParametrosArvore,data,Test_size,SIStrategyNum,OneHotStrat,GSscoring,max_depth
0,6,4,562,90,0.858006,0.600000,0.062500,0.113208,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 5, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 3, 'classifier__min_samples_split': 2, 'classifier__splitter': 'random'}","['X_all', 'y_all']",0.15,mean,ignore,precision,0.15
1,59,10,556,37,0.929003,0.855072,0.614583,0.715152,"{'classifier__criterion': 'gini', 'classifier__max_depth': 10, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 3, 'classifier__splitter': 'best'}","['X_all', 'y_all']",0.15,mean,ignore,precision,0.15
2,85,8,558,11,0.971299,0.913978,0.885417,0.899471,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 20, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'random'}","['X_all', 'y_all']",0.15,mean,ignore,precision,0.15
3,92,13,553,4,0.974320,0.876190,0.958333,0.915423,"{'classifier__criterion': 'gini', 'classifier__max_depth': 25, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'best'}","['X_all', 'y_all']",0.15,mean,ignore,precision,0.15
4,91,3,563,5,0.987915,0.968085,0.947917,0.957895,"{'classifier__criterion': 'gini', 'classifier__max_depth': 40, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'random'}","['X_all', 'y_all']",0.15,mean,ignore,precision,0.15
5,90,9,557,6,0.977341,0.909091,0.937500,0.923077,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 50, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 3, 'classifier__splitter': 'best'}","['X_all', 'y_all']",0.15,mean,ignore,precision,0.15
6,90,5,561,6,0.983384,0.947368,0.937500,0.942408,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 75, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 3, 'classifier__splitter': 'best'}","['X_all', 'y_all']",0.15,mean,ignore,precision,0.15
7,90,3,563,6,0.986405,0.967742,0.937500,0.952381,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 100, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 3, 'classifier__splitter': 'best'}","['X_all', 'y_all']",0.15,mean,ignore,precision,0.15
8,19,12,554,77,0.865559,0.612903,0.197917,0.299213,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 5, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 6, 'classifier__min_samples_split': 4, 'classifier__splitter': 'best'}","['X_all', 'y_all']",0.15,mean,ignore,recall,0.15
9,57,23,543,39,0.906344,0.712500,0.593750,0.647727,"{'classifier__criterion': 'gini', 'classifier__max_depth': 10, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 3, 'classifier__splitter': 'best'}","['X_all', 'y_all']",0.15,mean,ignore,recall,0.15


Após realizar o primeiro batch de testes, entendemos que seria necessário refinar as interações, visto que poderiamos eliminar alguns elimentos que foram interados.

Abaixo, um breve resumo das principais conclusões:

 * Não houve variação significativa em relação ao test size, portanto, adotamos como padrão o valor mais indicado na literatura (80/20 split). 
 * Chegamos à conclusão que o melhor método de scoring para o nosso problema, seria o f1, visto que o mesmo representa um equilibrio entre precision e recall. E no problema que estamos buscando resolver, um cenário de equilibrio é o mais desejado. 
 * Max_depth acima de 20 induz o modelo ao overfitting. E mesmo o valor 20 em algumas combinações, também ocasionou overfitting.
 * Após uma rodada de testes manuais (executando o modelo individualmente com os hiperparâmetros sinalizados pelo GridSearch), entendemos que seria necessário também olhar mais de perto as diferenças entre os resultados de treino e teste. Assim como no exemplo de max_depth acima de 20, encontramos alguns outros exemplos de overfitting quando olhamos também para os dados de treino.

##### 3.2.2 - Segundo batch de testes com diferentes hiperparametros

Para o segundo batch de testes, além dos pontos levantados acima, também buscamos na literatura algumas alternativas para controlar o overfitting.

Um delas, foi forçar a variação de sample_leaf e sample_split. Na literatura, encontramos referências indicando testes no valor entre 2 a aproximadamente 40 para sample_split e 1 entre aproximadamente 20, para sample_leaf.

Foi também reduzido o range de max_depth, com o objetivo de avaliar o range entre 10 e 20.

E por fim, incluímos nas interações, os testes com os dados de treino, para avaliar de maneira mais dinâmica possíveis overfitting. 

In [14]:
'''
#Combinação de parâmetros/hiperparâmetros a serem testados no treinamento do modelo
SIStrategyNum = ['mean','median','most_frequent']
max_depth=[[10],[13],[15],[16],[17],[18],[19],[20]]
train_test_data = [[X_all,y_all],[X_Sales,y_sales],[X_RandD,y_RandD]]

sample_split = []
for i in range(2,41,6):
    sample_split.append([i])

sample_leaf =  []
for i in range(1,20,3):
    sample_leaf.append([i])


#Nomeando os dataframes
X_all.name = 'X_all'
y_all.name = 'y_all'
X_Sales.name = 'X_Sales'
y_sales.name = 'y_sales'
X_RandD.name = 'X_RandD'
y_RandD.name = 'y_RandD'

            
#Separação das colunas numéricas continuas e categóricas que receberão diferentes tratativas no pipeline
numeric_features = df.drop(columns=['Attrition']).select_dtypes(exclude=[object]).columns.values.tolist()
categorical_features = df.select_dtypes(include=[object]).columns.values.tolist()

#Dicionário para armazenar os resultados
resumo_resultados = {
            'VP_teste': [],
            'FP_teste': [],
            'VN_teste': [],
            'FN_teste': [],
            'Acuracia_teste'  : [],
            'Precisao_teste'  : [], 
            'Revocação_teste' : [],
            'F-Medida_teste'  : [],
            'ParametrosArvore': [],
            'data':[],
            'SIStrategyNum': [],
            'max_depth':[],
            'VP_treino': [],
            'FP_treino': [],
            'VN_treino': [],
            'FN_treino': [],
            'Acuracia_treino'  : [],
            'Precisao_treino'  : [], 
            'Revocação_treino' : [],
            'F-Medida_treino'  : []
                     }

for data in train_test_data:
    for numstrat in SIStrategyNum:
        for depth in max_depth:
            for split in sample_split:
                for leaf in sample_leaf:
            
                    X_train, X_test, y_train, y_test = train_test_split(data[0],data[1], test_size=0.2, random_state=42)


                    cnts_pipeline = Pipeline([
                        ('impute', SimpleImputer(strategy=numstrat)),
                        ('scale', StandardScaler())])

                    categ_pipeline = Pipeline([
                        ('impute', SimpleImputer(strategy='most_frequent')),
                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

                    preprocess_pipeline = ColumnTransformer([
                        ('continuous', cnts_pipeline, numeric_features),
                        ('cat', categ_pipeline, categorical_features)])

                    pipeline = Pipeline(steps=[("preprocessor", preprocess_pipeline), ("classifier", DecisionTreeClassifier())])

                    param_grid = [{
                               'classifier__criterion': ['gini','entropy'],
                               'classifier__splitter': ['best', 'random'],
                               'classifier__max_depth': depth,
                               'classifier__max_features': ['auto', 'sqrt', 'log2'],
                               'classifier__min_samples_split':split,
                               'classifier__min_samples_leaf': leaf
                    }]

                    model_gs = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1')
                    model_gs.fit(X_train, y_train)

                    print("________________________________________________________________")
                    print('Modelo/Dados =',data[0].name,data[1].name)
                    print("Combinação de Parâmetros:")
                    print('NumericInputer =',numstrat, 'Max_depth =',depth)
                    print("*********************\n")
                    print("Resultados gerais:")
                    print('Best Score: ',model_gs.best_score_)
                    print('Melhores parâmetros da árvore: ',model_gs.best_params_)
                    print("*********************\n")
                    print("Matriz de Confusão:")
                    Results = {"train": {"actual": y_train,
                               "pred": model_gs.predict(X_train),
                               "prob": model_gs.predict_proba(X_train)},

                       "test": {"actual": y_test,
                               "pred": model_gs.predict(X_test),
                               "prob": model_gs.predict_proba(X_test)}}

                    resultados(Results)
                    print("________________________________________________________________")

                    y_pred = model_gs.predict(X_test)

                    vn, fp, fn, vp = Matriz_Confusao (y_test, y_pred)
                    Acuracia = calcacuracia(vp,fp,vn,fn)
                    P = calcprecisao(vp,fp,vn,fn)
                    R = calcrevocacao(vp,fp,vn,fn)
                    FMedida = calcfmedida(P,R)

                    y_pred = model_gs.predict(X_train)

                    vn_treino, fp_treino, fn_treino, vp_treino = Matriz_Confusao (y_train, y_pred)
                    Acuracia_treino = calcacuracia(vp_treino,fp_treino,vn_treino,fn_treino)
                    P_treino = calcprecisao(vp_treino,fp_treino,vn_treino,fn_treino)
                    R_treino = calcrevocacao(vp_treino,fp_treino,vn_treino,fn_treino)
                    FMedida_treino = calcfmedida(P_treino,R_treino)

                    resumo_resultados['VP_teste'].append(vp)
                    resumo_resultados['FP_teste'].append(fp)
                    resumo_resultados['VN_teste'].append(vn)
                    resumo_resultados['FN_teste'].append(fn)                
                    resumo_resultados['Acuracia_teste'].append(Acuracia)
                    resumo_resultados['Precisao_teste'].append(P)                
                    resumo_resultados['Revocação_teste'].append(R)
                    resumo_resultados['F-Medida_teste'].append(FMedida)                             
                    resumo_resultados['ParametrosArvore'].append(model_gs.best_params_)
                    resumo_resultados['data'].append([data[0].name,data[1].name])
                    resumo_resultados['SIStrategyNum'].append(numstrat)
                    resumo_resultados['max_depth'].append(depth)
                    resumo_resultados['VP_treino'].append(vp_treino)
                    resumo_resultados['FP_treino'].append(fp_treino)
                    resumo_resultados['VN_treino'].append(vn_treino)
                    resumo_resultados['FN_treino'].append(fn_treino)                
                    resumo_resultados['Acuracia_treino'].append(Acuracia_treino)
                    resumo_resultados['Precisao_treino'].append(P_treino)                
                    resumo_resultados['Revocação_treino'].append(R_treino)
                    resumo_resultados['F-Medida_treino'].append(FMedida_treino)                             


                    resultados_gerais2 = pd.DataFrame(resumo_resultados)

resultados_gerais2.to_csv('resultados_gerais2.csv', index = False)

Abaixo, uma demonstração de como ficou o DF exportado com os resultados e os hiperparâmetros de cada interação.

In [4]:
resultados_exemplo2 = pd.read_csv('resultados_gerais2.csv')
resultados_exemplo2.head(10)

,VP_teste,FP_teste,VN_teste,FN_teste,Acuracia_teste,Precisao_teste,Revocação_teste,F-Medida_teste,ParametrosArvore,data,SIStrategyNum,max_depth,VP_treino,FP_treino,VN_treino,FN_treino,Acuracia_treino,Precisao_treino,Revocação_treino,F-Medida_treino
0,73,32,709,68,0.886621,0.695238,0.517730,0.593496,"{'classifier__criterion': 'gini', 'classifier__max_depth': 10, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 4, 'classifier__splitter': 'best'}","['X_all', 'y_all']",mean,[10],360,47,2911,210,0.927154,0.884521,0.631579,0.736950
1,108,16,725,33,0.944444,0.870968,0.765957,0.815094,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 13, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'best'}","['X_all', 'y_all']",mean,[13],464,36,2922,106,0.959751,0.928000,0.814035,0.867290
2,118,7,734,23,0.965986,0.944000,0.836879,0.887218,"{'classifier__criterion': 'gini', 'classifier__max_depth': 15, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'best'}","['X_all', 'y_all']",mean,[15],516,12,2946,54,0.981293,0.977273,0.905263,0.939891
3,110,22,719,31,0.939909,0.833333,0.780142,0.805861,"{'classifier__criterion': 'gini', 'classifier__max_depth': 16, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'best'}","['X_all', 'y_all']",mean,[16],545,14,2944,25,0.988946,0.974955,0.956140,0.965456
4,110,13,728,31,0.950113,0.894309,0.780142,0.833333,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 17, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'random'}","['X_all', 'y_all']",mean,[17],518,14,2944,52,0.981293,0.973684,0.908772,0.940109
5,118,11,730,23,0.961451,0.914729,0.836879,0.874074,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 18, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'best'}","['X_all', 'y_all']",mean,[18],537,8,2950,33,0.988379,0.985321,0.942105,0.963229
6,123,8,733,18,0.970522,0.938931,0.872340,0.904412,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 19, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'best'}","['X_all', 'y_all']",mean,[19],567,0,2958,3,0.999150,1.000000,0.994737,0.997361
7,122,8,733,19,0.969388,0.938462,0.865248,0.900369,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 20, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'random'}","['X_all', 'y_all']",mean,[20],550,2,2956,20,0.993764,0.996377,0.964912,0.980392
8,84,17,724,57,0.916100,0.831683,0.595745,0.694215,"{'classifier__criterion': 'gini', 'classifier__max_depth': 10, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'best'}","['X_all', 'y_all']",median,[10],416,39,2919,154,0.945295,0.914286,0.729825,0.811707
9,92,9,732,49,0.934240,0.910891,0.652482,0.760331,"{'classifier__criterion': 'entropy', 'classifier__max_depth': 13, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__splitter': 'best'}","['X_all', 'y_all']",median,[13],448,14,2944,122,0.961451,0.969697,0.785965,0.868217


Os melhores resultados foram inputadores utilizados na Parte 3 do trabalho.